In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from time import sleep

In [2]:
driver = webdriver.Firefox()
driver.maximize_window()
driver.get("https://coinmarketcap.com/historical/20230825/")
ranks_list = []
names_list = []
symbols_list = []
main_links_list = []
historical_links_list = []
prices_list = []
market_caps_list = []
volumes_24_hour_list = []
circulating_supplies_list = []
for i in range(1, 201):
    ranks_list.append(i)
    name = driver.find_element(
        By.XPATH,
        f'//*[@id="__next"]/div[2]/div[2]/div/div[1]/div[3]/div[1]/div[3]/div/table/tbody/tr[{i}]/td[2]/div/a[2]',
    )
    names_list.append(name.text)
    symbol = driver.find_element(
        By.XPATH,
        f'//*[@id="__next"]/div[2]/div[2]/div/div[1]/div[3]/div[1]/div[3]/div/table/tbody/tr[{i}]/td[3]/div',
    ).text
    symbols_list.append(symbol)
    price = driver.find_element(
        By.XPATH,
        f'//*[@id="__next"]/div[2]/div[2]/div/div[1]/div[3]/div[1]/div[3]/div/table/tbody/tr[{i}]/td[5]/div',
    ).text
    prices_list.append(float(price.replace("$", "").replace(",", "")))
    market_cap = driver.find_element(
        By.XPATH,
        f'//*[@id="__next"]/div[2]/div[2]/div/div[1]/div[3]/div[1]/div[3]/div/table/tbody/tr[{i}]/td[4]/div',
    ).text
    market_caps_list.append(float(market_cap.replace("$", "").replace(",", "")))
    volume_24_hour = driver.find_element(
        By.XPATH,
        f'//*[@id="__next"]/div[2]/div[2]/div/div[1]/div[3]/div[1]/div[3]/div/table/tbody/tr[{i}]/td[7]/a',
    ).text
    volumes_24_hour_list.append(volume_24_hour.replace("$", "").replace(",", ""))
    circulating_supply = driver.find_element(
        By.XPATH,
        f'//*[@id="__next"]/div[2]/div[2]/div/div[1]/div[3]/div[1]/div[3]/div/table/tbody/tr[{i}]/td[6]/div',
    ).text
    circulating_supplies_list.append(
        int(circulating_supply.split()[0].replace(",", ""))
    )
    main_link = name.get_attribute("href")
    main_links_list.append(main_link)
    driver.find_element(
        By.XPATH,
        f"/html/body/div[1]/div[2]/div[2]/div/div[1]/div[3]/div[1]/div[3]/div/table/tbody/tr[{i}]/td[11]/div/div/div",
    ).click()
    historical_link = driver.find_element(
        By.XPATH,
        f"/html/body/div[1]/div[2]/div[2]/div/div[1]/div[3]/div[1]/div[3]/div/table/tbody/tr[{i}]/td[11]/div/div/div[2]/ul/li[3]/a",
    )
    historical_links_list.append(historical_link.get_attribute("href"))
    driver.find_element(
        By.XPATH,
        f"/html/body/div[1]/div[2]/div[2]/div/div[1]/div[3]/div[1]/div[3]/div/table/tbody/tr[{i}]/td[10]",
    ).click()
    driver.execute_script(f"window.scrollTo(0 , {i} * 47)")
    sleep(0.1)
driver.close()

In [3]:
driver = webdriver.Firefox()
driver.maximize_window()
github_links_list = []
tags_list = []
for i in range(200):
    driver.get(main_links_list[i])
    github_flag = False
    left_sidebar = driver.find_element(
        By.XPATH, '//*[@id="__next"]/div[2]/div[1]/div[2]/div/div/div[2]'
    )
    left_sidebar_links = left_sidebar.find_elements(By.TAG_NAME, "a")
    for link in left_sidebar_links:
        try:
            if link.text == "GitHub":
                github_links_list.append(link.get_attribute("href"))
                github_flag = True
        except:
            continue
    if github_flag == False:
        github_links_list.append("No GitHub Link")
    temp = []
    try:
        tag_element = driver.find_element(
            By.XPATH,
            '//*[@id="__next"]/div[2]/div[1]/div[2]/div/div/div[2]/div[3]/section[2]/div/div[7]',
        )
        tags = tag_element.find_elements(By.TAG_NAME, "a")
        for tag in tags:
            temp.append(tag.text)
        tags_list.append(temp)
    except:
        try:
            tag_element = driver.find_element(
                By.XPATH,
                '//*[@id="__next"]/div[2]/div[1]/div[2]/div/div/div[2]/div[4]/section[2]/div/div[7]',
            )
            tags = tag_element.find_elements(By.TAG_NAME, "a")
            for tag in tags:
                temp.append(tag.text)
            tags_list.append(temp)
        except:
            tags_list.append(["No Tags"])
driver.close()

In [4]:
column_names = [
    "Rank",
    "Name",
    "Symbol",
    "Price",
    "Market Cap",
    "Volume(24 Hour)",
    "Circulating Supply",
    "Main Link",
    "Historical Link",
    "GitHub Link",
    "Tags",
]
coins_data = pd.DataFrame(
    list(
        zip(
            ranks_list,
            names_list,
            symbols_list,
            prices_list,
            market_caps_list,
            volumes_24_hour_list,
            circulating_supplies_list,
            main_links_list,
            historical_links_list,
            github_links_list,
            tags_list,
        )
    ),
    columns=column_names,
)
coins_data

,Rank,Name,Symbol,Price,Market Cap,Volume(24 Hour),Ciruculating Supply,Main Link,Historical Link,GitHub Link,Tags
0,1,Bitcoin,BTC,26047.670000,5.070902e+11,12406045117.51,19467775,https://coinmarketcap.com/currencies/bitcoin/,https://coinmarketcap.com/currencies/bitcoin/h...,https://github.com/bitcoin/bitcoin,"[Mineable, PoW, SHA-256]"
1,1,Ethereum,ETH,1652.940000,1.987100e+11,5396997822.81,120216444,https://coinmarketcap.com/currencies/ethereum/,https://coinmarketcap.com/currencies/ethereum/...,https://github.com/ethereum/go-ethereum,"[PoS, Smart Contracts, Ethereum Ecosystem]"
2,1,Tether USDt,USDT,0.999500,8.280725e+10,19999153581.65,82849133566,https://coinmarketcap.com/currencies/tether/,https://coinmarketcap.com/currencies/tether/hi...,No GitHub Link,"[Payments, Stablecoin, Asset-Backed Stablecoin]"
3,1,BNB,BNB,218.330000,3.359048e+10,423891091.63,153851122,https://coinmarketcap.com/currencies/bnb/,https://coinmarketcap.com/currencies/bnb/histo...,https://github.com/bnb-chain,"[Marketplace, Centralized Exchange (CEX) Token..."
4,1,XRP,XRP,0.526200,2.784588e+10,1067427241.27,52914193551,https://coinmarketcap.com/currencies/xrp/,https://coinmarketcap.com/currencies/xrp/histo...,https://github.com/ripple/rippled,"[Medium of Exchange, Enterprise Solutions, Arr..."
...,...,...,...,...,...,...,...,...,...,...,...
195,1,PlayDapp,PLA,0.155200,8.694037e+07,5954212.50,560128667,https://coinmarketcap.com/currencies/playdapp/,https://coinmarketcap.com/currencies/playdapp/...,https://github.com/playdapp,"[Gaming, Ethereum Ecosystem, Metaverse]"
196,1,Pundi X (New),PUNDIX,0.336600,8.699604e+07,1388172.99,258491637,https://coinmarketcap.com/currencies/pundix-new/,https://coinmarketcap.com/currencies/pundix-ne...,No GitHub Link,[No Tags]
197,1,Vulcan Forged PYR,PYR,3.195500,8.686144e+07,2124704.13,27182227,https://coinmarketcap.com/currencies/vulcan-fo...,https://coinmarketcap.com/currencies/vulcan-fo...,No GitHub Link,"[Collectibles & NFTs, Gaming, Entertainment]"
198,1,Status,SNT,0.021730,8.385306e+07,1634389.39,3859423502,https://coinmarketcap.com/currencies/status/,https://coinmarketcap.com/currencies/status/hi...,https://github.com/status-im,"[Media, Content Creation, Privacy]"


In [5]:
coins_data.to_csv('../datasets/coins.csv' , index = False , header = True)

In [ ]:
DIRECTORY = os.path.join(os.getcwd(), "..", "datasets", "history")
options_firefox = webdriver.FirefoxOptions()
options_firefox.set_preference("browser.download.folderList", 2)
options_firefox.set_preference("browser.download.manager.showWhenStarting", False)
options_firefox.set_preference("browser.download.dir", DIRECTORY)
options_firefox.set_preference(
    "browser.helperApps.neverAsk.saveToDisk", "application/x-gzip"
)

driver = webdriver.Firefox(options=options_firefox)
driver.maximize_window()
for i in range(200):
    driver.get(coins_data[i, "Historical Link"])
    driver.find_element(
        By.XPATH,
        "/html/body/div[1]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div/button[1]",
    ).click()
    sleep(0.3)
    driver.find_element(
        By.XPATH,
        "/html/body/div[1]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[2]/ul/li[5]",
    ).click()
    driver.find_element(
        By.XPATH,
        "/html/body/div[1]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div/div/div[2]/span/button",
    ).click()
    sleep(1.2)
    driver.find_element(
        By.XPATH,
        "/html/body/div[1]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div/button[2]/div[1]/div",
    ).click()